In [ ]:
import copy
import json
from os.path import abspath, dirname
from os.path import join as pjoin
from zipfile import ZipFile

from pymedphys._imports import numpy as np
from pymedphys._imports import pydicom, pytest

import pymedphys
from pymedphys._data import download
from pymedphys._dicom import collection, create, dose, orientation
from pymedphys._gamma.api import gamma_dicom

In [ ]:
def get_file_within_data_zip(zip_name, file_name):
    dose_data_files = pymedphys.zip_data_paths(zip_name)
    path_match = [path for path in dose_data_files if path.name == file_name]

    assert len(path_match) == 1

    return str(path_match[0])

def get_data_file(orientation_key):
    return download.get_file_within_data_zip(
        "dicomorient_doses_only.zip",
        "RD.DICOMORIENT.Dose_{}.dcm".format(orientation_key),
    )

In [ ]:
ORIENTATIONS_SUPPORTED = ["FFDL", "FFDR", "FFP", "FFS", "HFDL", "HFDR", "HFP", "HFS"]

In [ ]:
test_ds_dict = {
        key: pydicom.dcmread(get_data_file(key))
        for key in ORIENTATIONS_SUPPORTED
    }

In [ ]:
gammas_by_orient = dict.fromkeys(ORIENTATIONS_SUPPORTED)

for orient, ds in test_ds_dict.items():

    try:
        gammas_by_orient[orient] = gamma_dicom(ds, ds, 1, 1, max_gamma=1.1, random_subset=100)
        print(f"{orient}: {gammas_by_orient[orient].shape}")

    except ValueError as e:
        print(f"{orient}: {e}")